In [9]:
import backtrader as bt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pytz

def load_and_prepare_data(file_path):
    data = pd.read_csv(file_path, sep='\t')
    data['datetime'] = pd.to_datetime(data['<DATE>'] + ' ' + data['<TIME>'])
    data = data[['datetime', '<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>', '<TICKVOL>']]
    data = data.sort_values('datetime')
    
    data['datetime'] = data['datetime'].dt.tz_localize('UTC').dt.tz_convert('America/New_York')
    
    data = data[(data['datetime'].dt.hour >= 12) & (data['datetime'].dt.hour < 21)]
    data = data.reset_index(drop=True)
    
    return data

class SuperTrend(bt.Indicator):
    lines = ('supertrend',)
    params = (('period', 10), ('multiplier', 1.5))
    
    def __init__(self):
        self.addminperiod(self.params.period)
        self.atr = bt.indicators.AverageTrueRange(self.data, period=self.params.period)
        self.lines.supertrend = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.period) 

    def next(self):
        if len(self) < self.params.period:
            return

        atr = self.atr[0]
        hl2 = (self.data.high[0] + self.data.low[0]) / 2

        if self.lines.supertrend[0] is None:
            self.lines.supertrend[0] = hl2 - (self.params.multiplier * atr)

        if self.data.close[0] > self.lines.supertrend[-1]:
            self.lines.supertrend[0] = max(self.lines.supertrend[-1], hl2 - (self.params.multiplier * atr))
        else:
            self.lines.supertrend[0] = min(self.lines.supertrend[-1], hl2 + (self.params.multiplier * atr))

class ComprehensiveStrategy(bt.Strategy):
    params = (
        ('sma_period', 20),
        ('ema_period', 20),
        ('rsi_period', 14),
        ('macd_period', (12, 26, 9)),
        ('bollinger_period', 20),
        ('bollinger_dev', 2),
        ('stochastic_period', 14),
        ('adx_period', 14),
        ('cci_period', 14),
        ('williams_period', 14),
        ('atr_period', 14),
        ('supertrend_period', 10),
        ('supertrend_multiplier', 1.5),
        ('reward_risk_ratio', 3),
        ('position_size', 0.1),
        ('stop_loss_percent', 0.02),
        ('take_profit_percent', 0.05),
    )

    def __init__(self):
        self.sma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.sma_period)
        self.ema = bt.indicators.ExponentialMovingAverage(self.data.close, period=self.params.ema_period)
        self.rsi = bt.indicators.RelativeStrengthIndex(self.data.close, period=self.params.rsi_period)
        self.macd = bt.indicators.MACD(self.data.close, period_me1=self.params.macd_period[0], 
                                       period_me2=self.params.macd_period[1], period_signal=self.params.macd_period[2])
        self.bollinger = bt.indicators.BollingerBands(self.data.close, period=self.params.bollinger_period, devfactor=self.params.bollinger_dev)
        self.stochastic = bt.indicators.Stochastic(self.data)
        self.adx = bt.indicators.AverageDirectionalMovementIndex(self.data, period=self.params.adx_period)
        self.cci = bt.indicators.CommodityChannelIndex(self.data, period=self.params.cci_period)
        self.williams = bt.indicators.WilliamsR(self.data, period=self.params.williams_period)
        self.atr = bt.indicators.AverageTrueRange(self.data, period=self.params.atr_period)
        self.supertrend = SuperTrend(period=self.params.supertrend_period, multiplier=self.params.supertrend_multiplier)

        self.order = None
        self.capital_values = []
        self.dates = []

    def next(self):
        if self.order:
            return  
        
        self.dates.append(self.data.datetime.date(0))
        self.capital_values.append(self.broker.getvalue())

        price = self.data.close[0]

        #خرید
        if (self.rsi[0] >= 40 and self.rsi[-1] < 40 and
            self.supertrend[0] < price and price > self.ema[0]):
            sl = price - self.atr[0]
            tp = price + (sl - price) * 5
            size = self.params.position_size
            self.order = self.buy(size=size)
            self.sell(exectype=bt.Order.Stop, price=sl, size=size)
            self.sell(exectype=bt.Order.Limit, price=tp, size=size)
        
        # فروش
        elif (self.rsi[0] < 40 and self.rsi[-1] >= 40 and
              self.supertrend[0] > price and price < self.ema[0]):
            sl = price + self.atr[0]
            tp = price - (price - sl) * 5
            size = self.params.position_size
            self.order = self.sell(size=size)
            self.buy(exectype=bt.Order.Stop, price=sl, size=size)
            self.buy(exectype=bt.Order.Limit, price=tp, size=size)

    def notify_order(self, order):
        if order.status in [order.Completed, order.Canceled, order.Margin]:
            self.order = None

def prepare_data_feed(data):
    return bt.feeds.PandasData(
        dataname=data,
        datetime='datetime',
        open='<OPEN>',
        high='<HIGH>',
        low='<LOW>',
        close='<CLOSE>',
        volume='<TICKVOL>',
        openinterest=-1
    )

def run_backtest(file_path):
    data = load_and_prepare_data(file_path)
    data_feed = prepare_data_feed(data)

    cerebro = bt.Cerebro()
    cerebro.addstrategy(ComprehensiveStrategy)
    cerebro.adddata(data_feed)
    cerebro.broker.set_cash(1000) 

    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe_ratio')

    results = cerebro.run()
    strat = results[0]

    total_value = cerebro.broker.getvalue()
    print(f'پایان بک‌تست: ارزش نهایی حساب: {total_value}')
    print(f'میزان رشد یا ریزش حساب: {total_value - cerebro.broker.startingcash}')

    fig, ax = plt.subplots()
    ax.plot(strat.dates, strat.capital_values, label='Profit')
    ax.set_xlabel('time')
    ax.set_ylabel('Balance')
    ax.set_title('Hashem BackTest')
    ax.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('بک تست.png')
    plt.close()

if __name__ == '__main__':
    file_path = 'C:/Users/nima/Desktop/کتابخونه طلایی/XAUUSD.csv'
    run_backtest(file_path)


پایان بک‌تست: ارزش نهایی حساب: 1061.6837744037377
میزان رشد یا ریزش حساب: 61.68377440373774
